In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler
import pickle

In [2]:
df=pd.read_csv("Churn_Modelling.csv")

In [3]:
df=df.drop(columns=["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
#Encoding the categorical column using LabelEncoder.
Labelencoder=LabelEncoder()
df["Gender"]=Labelencoder.fit_transform(df["Gender"])
df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1686,682,Spain,1,36,5,0.00,2,1,1,147758.51,0
6442,491,France,1,33,5,83134.30,1,1,0,187946.55,0
1169,653,France,0,38,8,102133.38,1,1,1,166520.96,0
1564,773,Germany,0,42,8,152324.66,2,1,0,171733.22,0
639,785,France,0,36,4,135438.40,1,0,0,190627.01,0


In [5]:
#Encoding Geography using One hot encoder.
OneHotEncoder_geo=OneHotEncoder(sparse_output=False)
df_geo=OneHotEncoder_geo.fit_transform(df[["Geography"]])

In [6]:
df_geo=pd.DataFrame(df_geo,columns=OneHotEncoder_geo.get_feature_names_out(["Geography"]))
df=pd.concat([df.drop("Geography",axis=1),df_geo],axis=1)

In [7]:
#divide input and output
X=df.drop("Exited",axis=1)
Y=df["Exited"]

#divide the train and test data

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=23)

#scale the train and test data

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [8]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(Labelencoder,file)

with open('One_Hot_Encoder_geo.pkl','wb') as file:
    pickle.dump(OneHotEncoder_geo,file)

with open('StandScaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [10]:
#define the model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

In [11]:
#compile the model
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
#Setup TensorBoard forlogs
log_dir='log/file/'+ datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensor_board_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [13]:
#setup earlystopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [14]:
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),
    epochs=100,callbacks=[tensor_board_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 4ms/step - loss: 0.4508 - accuracy: 0.8084 - val_loss: 0.3937 - val_accuracy: 0.8415
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3802 - accuracy: 0.8460 - val_loss: 0.3519 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3515 - accuracy: 0.8575 - val_loss: 0.3446 - val_accuracy: 0.8585
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3428 - accuracy: 0.8626 - val_loss: 0.3397 - val_accuracy: 0.8625
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3390 - accuracy: 0.8622 - val_loss: 0.3361 - val_accuracy: 0.8640
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3339 - accuracy: 0.8635 - val_loss: 0.3348 - val_accuracy: 0.8640
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3325 - accuracy: 0.8644 - val_loss: 0.3355 - val_accuracy: 0.85

In [15]:
model.save('model.h5')

c:\Users\W540\Desktop\ANNClassificationNew\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
